In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf

%matplotlib inline
#%matplotlib tk

In [2]:
k = 3.0
vbar = 0.1
v0 = 0.08
rho = -0.8
sigma = 0.25

S0 = 100.0
r = 0.02
K =  7.660333466800909
tau = 0.01917808
tbar = 30/365.0

In [27]:
z = np.linspace(0.5, 100, 1000) * 1j + 2.0
z = np.array([z])
u = np.array([0.0122236989606157641980521, 0.0366637909687334933302153,     
                    0.0610819696041395681037870, 0.0854636405045154986364980,     
                    0.1097942311276437466729747, 0.1340591994611877851175753,     
                    0.1582440427142249339974755, 0.1823343059853371824103826,     
                    0.2063155909020792171540580, 0.2301735642266599864109866,     
                    0.2538939664226943208556180, 0.2774626201779044028062316,     
                    0.3008654388776772026671541, 0.3240884350244133751832523,     
                    0.3471177285976355084261628, 0.3699395553498590266165917,     
                    0.3925402750332674427356482, 0.4149063795522750154922739,     
                    0.4370245010371041629370429, 0.4588814198335521954490891,     
                    0.4804640724041720258582757, 0.5017595591361444642896063,     
                    0.5227551520511754784539479, 0.5434383024128103634441936,     
                    0.5637966482266180839144308, 0.5838180216287630895500389,     
                    0.6034904561585486242035732, 0.6228021939105849107615396,     
                    0.6417416925623075571535249, 0.6602976322726460521059468,     
                    0.6784589224477192593677557, 0.6962147083695143323850866,     
                    0.7135543776835874133438599, 0.7304675667419088064717369,     
                    0.7469441667970619811698824, 0.7629743300440947227797691,     
                    0.7785484755064119668504941, 0.7936572947621932902433329,     
                    0.8082917575079136601196422, 0.8224431169556438424645942,     
                    0.8361029150609068471168753, 0.8492629875779689691636001,     
                    0.8619154689395484605906323, 0.8740527969580317986954180,     
                    0.8856677173453972174082924, 0.8967532880491581843864474,     
                    0.9073028834017568139214859, 0.9173101980809605370364836,     
                    0.9267692508789478433346245, 0.9356743882779163757831268,     
                    0.9440202878302201821211114, 0.9518019613412643862177963,     
                    0.9590147578536999280989185, 0.9656543664319652686458290,     
                    0.9717168187471365809043384, 0.9771984914639073871653744,                          
                    0.9820961084357185360247656, 0.9864067427245862088712355,     
                    0.9901278184917343833379303, 0.9932571129002129353034372,     
                    0.9957927585349811868641612, 0.9977332486255140198821574,     
                    0.9990774599773758950119878, 0.9998248879471319144736081])

lb = 0
ub = 150
mid = 0.5 * (ub + lb)
halfRange = 0.5 * (ub - lb)
up_u = mid + u * halfRange
down_u = np.flip(mid - u * halfRange)
all_u = np.append(down_u, up_u)

In [4]:
def CFvol(u, tau):
    G = np.cosh(k*tau*0.5) + (1 - sigma**2 * 1j * u/k)*np.sinh(k*tau*0.5)
    F = (v0*1j*u/G) * np.exp(-k*tau*0.5)
    CF = (np.exp(k*tau*0.5)/G) ** (2.0*k*vbar/(sigma**2)) * np.exp(F)
    return CF, G, F

In [5]:
def VIXintegrand(u, tau, K, tbar):
    aTbar = (1 - np.exp(-tbar*k))/k
    U = -u/tbar * aTbar
    bTbar = vbar*(tbar - aTbar)
    CF, G, F = CFvol(U, tau)
    part1 = np.exp(-1j*u/tbar*bTbar)
    part2 = 1 - erf(K*(-1j*u)**0.5 /S0)
    part3 = (-1j*u)**1.5
    integrand = CF * part1*part2/part3

    return integrand.real, integrand, aTbar, G, F, U, 1j*u

In [6]:
def gradVIX(u, tau, K, tbar):
    reInt, rawInt, aTbar, G, F, U, iu = VIXintegrand(u, tau, K, tbar)
    var = sigma ** 2
    tmp1 = 2.0 * k / var
    tmp2 = k * tau * 0.5
    tmp3 = np.exp(tmp2)
    tmp4 = G ** 2
    iU = 1j * U
    iuOverTbar = iu/tbar
    
    G_sigma = -2.0 * sigma * iU/k*np.sinh(tmp2)
    
    h_v0 = F/v0
    h_vbar = tmp1 * np.log(tmp3/G)
    h_sigma = -2.0*vbar/sigma*h_vbar - tmp1*vbar/G*G_sigma - v0*iU/(tmp4*tmp3)*G_sigma
    h_k = -sigma/(2.0*k)*h_sigma + vbar*tau*iU/(G*tmp3) - v0*U*tau/(2.0*k*tmp4)*(2.0*k*1j + U*var)
    
    atauBar_k = (tbar - aTbar*(k*tbar + 1))/k
    btauBar_vbar = tbar - aTbar
    btauBar_k = -vbar*atauBar_k
    
    G_U = (G - tmp3)/U
    hPrime = h_k - u/tbar*(F/U - 1.0/G*(tmp1*vbar + F)*G_U)*atauBar_k
    H_vbar = h_vbar - iuOverTbar*btauBar_vbar
    H_k = hPrime - iuOverTbar*btauBar_k
    
    return (H_k*rawInt).real, (H_vbar*rawInt).real, (h_v0*rawInt).real, (h_sigma*rawInt).real
    

In [29]:
#%matplotlib tk
uu = np.array(np.linspace(-150, 150, 10000))
reIntegrand, _, _, _, _, _, _  = VIXintegrand(uu+5*1j, tau, K, tbar)
reInte, _, _, _, _, _, _  = VIXintegrand(all_u+5*1j, tau, K, tbar)
plt.figure()
plt.plot(uu, reIntegrand, 'r')
plt.scatter(all_u, reInte)
plt.title("Integrand of VIX pricing (Eq (3.14)) as a function of u")

Text(0.5,1,'Integrand of VIX pricing (Eq (3.14)) as a function of u')

In [15]:
plt.figure()
reIntegrandDiffIm, _, _, _, _, _, _  = VIXintegrand(z, tau, K, tbar)
plt.plot((z*(-1j)).real.flatten(), reIntegrandDiffIm.flatten())

In [25]:
us = np.array(np.linspace(-300, 300, 3000))

gradk, gradvbar, gradv0, gradsigma = gradVIX(all_u+5*1j, tau, K, tbar)\

gradk5, gradvbar5, gradv05, gradsigma5 = gradVIX(us+5*1j, tau, K, tbar)

In [26]:
plt.figure()
plt.plot(us, gradk5, 'r')
plt.scatter(all_u, gradk)
plt.title("Integrand of Eq (3.28) k")
plt.figure()
plt.plot(us, gradvbar5, 'b')
plt.scatter(all_u, gradvbar)
plt.title("Integrand of Eq (3.28) vbar")
plt.figure()
plt.plot(us, gradv05, 'y')
plt.scatter(all_u, gradv0)
plt.title("Integrand of Eq (3.28) v0")
plt.figure()
plt.plot(us, gradsigma5, 'k')
plt.scatter(all_u, gradsigma)
plt.title("Integrand of Eq (3.28) sigma")

Text(0.5,1,'Integrand of Eq (3.28) sigma')

In [129]:
plt.figure()
plt.scatter(all_u, gradsigma10)

In [132]:
plt.figure()
plt.scatter(all_u, gradk10)
plt.scatter(all_u, gradk5)
plt.figure()
plt.scatter(all_u, gradvbar10)
plt.scatter(all_u, gradvbar5)
plt.figure()
plt.scatter(all_u, gradv010)
plt.scatter(all_u, gradv05)
plt.figure()
plt.scatter(all_u, gradsigma10)
plt.scatter(all_u, gradsigma5)